In [1]:
import random
from greedy import greedy_cit

In [70]:
class TestSuite():
    def __init__(self, base, additional):
        self.base = base
        self.additional = additional
        self.fitness_value = None
        
    def assign_fitness_value(self, fitness_value):
        self.fitness_value = fitnes_value
        
    def make_test_suite(self):
        self.test_suite = self.base + self.additional
        
    def eval_variety(self, parameter_num, parameter_range):
        appear_time = []
        
        for ith_param in range(parameter_num):
            value_list = [1 for _ in range(parameter_range+1)]
            appear_time.append(value_list)
            
        for testcase in self.additional:
            for ind in range(len(testcase)):
                appear_time[ind][testcase[ind]] += 1
                
        score = 1
        for ith_param in appear_time:
            for ind in range(len(ith_param)):
                #if ith_param[ind] != 1:
                    score = score * ((ith_param[ind]*parameter_num)/len(self.additional))
                    #prevent the fitness value become too big or too small for temprorary
                    #if there are better standard, need to modify
                
        self.fitness_value = score
    
    #assume soft_list format [constraint1 = [[parameter index,lower range, upper range], [], ...], constraint2 = []...]
    #if test case satisfy constraint condition, multiply fitness value 1.3
    def eval_soft_constraint(self, soft_list):
        for constraint in soft_list:
            for testcase in self.additional:
                for (ind, condition) in enumerate(constraint):
                    if testcase[condition[0]] < condition[1] or condition[2] < testcase[condition[0]]:
                        break
                    if ind == (len(constraint)-1):
                        self.fitness_value = self.fitness_value * 1.3
                    
        
    #hard_list format is same as soft_list
    #if test case satisfy hard constraint condition, set fitness value to 0 and return
    def eval_hard_constraint(self, hard_list):
        for constraint in hard_list:
            for testcase in self.additional:
                for (ind, condition) in enumerate(constraint):
                    if testcase[condition[0]] < condition[1] or condition[2] < testcase[condition[0]]:
                        break
                    if ind == (len(constraint)-1):
                        self.fitness_value = 0
                        return
                
    def print_fit_val(self):
        print(self.fitness_value)
                

In [71]:
class GA():
    # Range for budget
    # len(base) < budget == len(base) + len(additional) < len(greedy_suite)
    def __init__(self, greedy_suite, base, parameter_num, parameter_range, budget, population_num, mutation_rate):
        self.greedy_suite = greedy_suite
        self.base = base
        self.parameter_num = parameter_num
        # maximum value of the parameter
        self.parameter_range = parameter_range
        self.budget = budget
        self.mutation_rate = mutation_rate
        self.population_num = population_num
        self.fitness_result =[]
        self.population = []
        
        
    # must do this
    def delete_base_from_greedy(self):
        for i in range(len(self.base)):
            target = self.base[i]
            if(target in self.greedy_suite):
                self.greedy_suite.remove(target)
    # 수정 필요
    def initial_population(self):
        #Using Approach 1: randomly choosing from the greedy - test suite
        initial_population = []
        for i in range(self.population_num):
            gene = random.choices(self.greedy_suite, k = self.budget - len(self.base))
            new_suite = TestSuite(self.base, gene)
            initial_population.append(new_suite)
        self.population = initial_population
            
    def fitness_calculate(self, soft_list, hard_list):
        
        #print("fitness calculating this population")
        #for i in range(len(self.population)):
        #    print(self.population[i].additional)
        self.fitness_result = []
        for i in range(self.population_num):
            self.population[i].eval_variety(self.parameter_num, self.parameter_range)
            self.population[i].eval_soft_constraint(soft_list)
            self.population[i].eval_hard_constraint(hard_list)
            self.fitness_result.append(self.population[i].fitness_value)
    
    def select_parent(self):
        result = random.choices(self.population, weights = self.fitness_result, k = 2)

        return result
    
    
    def make_children(self, parent1, parent2):
        #Sort the list lexicographically
        # For now, just take half / half
        sorted_parent1 = sorted(parent1.additional, key = lambda elem: elem)
        sorted_parent2 = sorted(parent2.additional, key = lambda elem: elem)
        
        child1 = []
        child2 = []


        assert(self.budget - len(self.base) == len(parent1.additional))
        assert(self.budget - len(self.base) == len(parent2.additional))
        additional_length = self.budget - len(self.base)
        for i in range(additional_length):
            if(i < int(additional_length / 2)):
                child1.append(sorted_parent1[i])
                child2.append(sorted_parent2[i])
            else:
                child1.append(sorted_parent2[i])
                child2.append(sorted_parent1[i])
                
                
        return child1
    
    def mutate_test_suite(self, test_suite):
        #iterate for each parameter
        mutant = []
        for i in range(len(test_suite[0])):
            new_dict = {}
            for j in range(len(test_suite)):
                if(test_suite[j][i] not in new_dict.keys()):
                    new_dict[test_suite[j][i]] = 0
                else:
                    new_dict[test_suite[j][i]] = new_dict[test_suite[j][i]] + 1
            minimum_cnt = -1
            minimum_var = None
            for j in range(len(new_dict.keys())):
                temp = list(new_dict.keys())[j]
                temp_cnt = new_dict[temp]
                if(minimum_cnt < 0 or temp_cnt < minimum_cnt):
                    minimum_cnt = temp_cnt
                    minimum_var = temp
            
            mutant.append(minimum_var)
        del test_suite[0]
        test_suite.append(mutant)
        return test_suite
        
    def mutate_population(self, new_population):
        mutated_population = []
        for i in range(len(new_population)):
            target_test_suite = new_population[i].additional
            if(random.random() < self.mutation_rate):
                new_test_suite = self.mutate_test_suite(target_test_suite)
                mutated_population.append(TestSuite(self.base, new_test_suite))
            else:
                mutated_population.append(TestSuite(self.base, target_test_suite))
                
        return mutated_population
                
    def next_generation(self):

        elite_num = int(self.population_num * 4 / 10)

        children_num = int(self.population_num * 4 / 10)

        random_children_num = self.population_num - elite_num - children_num

        
        sorted_population = sorted(self.population, key = lambda test_suite: test_suite.fitness_value)   
        elite_children = sorted_population[self.population_num - elite_num:]
        print(sorted_population[-1].fitness_value)
       # for i in range(len(sorted_population)):
        #    print(sorted_population[i].fitness_value)

        
        children = []
        for i in range(children_num):
            [parent1, parent2] = self.select_parent()
            child = self.make_children(parent1, parent2)
            children.append(TestSuite(self.base, child))


        
        random_children = []
        for i in range(random_children_num):
            random_child = random.choice(self.population)
            random_children.append(random_child)
            

        new_population = elite_children + children + random_children
        
        mutated_population = self.mutate_population(new_population)
        
    #    print("previous generation")
    #    for i in range(len(self.population)):
    #        print(self.population[i].additional)
    #    print("next_generation")
    #    for i in range(len(self.population)):
    #        print(new_population[i].additional)
        
        self.population = mutated_population
        
    def adjust(self):

        for i in range(len(self.population)):
            target_test_suite = self.population[i].additional
            target_set = set(tuple(element) for element in target_test_suite)
            target_lst = [list(t) for t in set(tuple(element) for element in target_set)]
            need_cnt = len(target_test_suite) - len(target_lst)
            for j in range(need_cnt):
                new_case = random.choice(self.greedy_suite)
                while(new_case in target_lst):
                    new_case = random.choice(self.greedy_suite)
                target_lst.append(new_case)
            
            self.population[i].additional = target_lst
            
    
    

In [72]:
a = (greedy_cit(2, ["a1", "a2", "a3", "a4", "a5"], [10, 10, 10, 10, 10]))                 
print(len(a))


146


In [73]:
b = (greedy_cit(3, ["a1", "a2", "a3", "a4", "a5"], [10, 10, 10, 10, 10]))     
print(len(b))                                                  

1598


In [74]:
temp = GA(b, a, 5, 10, 900, 800, 0.7)
print(len(a))
print(len(b))

146
1598


In [75]:
temp.delete_base_from_greedy()

In [76]:
temp.initial_population()

In [77]:


for i in range(100):
    print(i)
    temp.fitness_calculate([],[])
    temp.next_generation()
    temp.adjust()


0
1.1505744712052558e-26
1
1.2965690589604987e-26
2
1.30814351803589e-26
3
1.3125474775181857e-26
4
1.3225732599957198e-26
5
1.369267914223392e-26
6
1.373664098512395e-26
7
1.414991749433204e-26
8
1.437372007949802e-26
9
1.4590268577513278e-26
10
1.4590268577513278e-26
11
1.496408491109442e-26
12
1.518126372196748e-26
13
1.52148555593222e-26
14
1.542966626966214e-26
15
1.5547132251620638e-26
16
1.5578741144809656e-26
17
1.5638930924681357e-26
18
1.5836124458485712e-26
19
1.6013278909179148e-26
20
1.6369544189085896e-26
21
1.6552524389319248e-26
22
1.6586450621288334e-26
23
1.6884778990650465e-26
24
1.6999164901380232e-26
25
1.7022780354334827e-26
26
1.7182396207373093e-26
27
1.7235331514058187e-26
28
1.7424247751290494e-26
29
1.7495871718920463e-26
30
1.7712746018133426e-26
31
1.773632892806117e-26
32
1.7844353170594464e-26
33
1.792179108710275e-26
34
1.8032269431709496e-26
35
1.8130814984577908e-26
36
1.8288836037749356e-26
37
1.8463230437407707e-26
38
1.863418935994096e-26
39
1.86505

In [78]:
temp.fitness_calculate([],[])
sorted_population = sorted(temp.population, key = lambda test_suite: test_suite.fitness_value)   
with open('gentic_cit.txt', 'w') as f:
    f.write(str(sorted_population[-1].base + sorted_population[-1].additional))
print(len(sorted_population[-1].base + sorted_population[-1].additional))

900


Random 

In [79]:
random_population = random.choices(b, k = 900)
with open('random.txt', 'w') as f:
    f.write(str(random_population))
print(len(random_population))

900


Just GA
self, greedy_suite, base, parameter_num, parameter_range, budget, population_num, mutation_rate):

In [80]:
temp2 = GA(b, [], 5, 10, 900, 800, 0.7)
temp2.delete_base_from_greedy()
temp2.initial_population()

In [ ]:
for i in range(100):
    print(i)
    temp2.fitness_calculate([],[])
    temp2.next_generation()
    temp2.adjust()


0
4.34240533758695e-27
1
4.5856659795789586e-27
2
4.894315082856555e-27
3
4.9321189623579785e-27
4
4.927376554534314e-27
5
4.9314495775115015e-27
6
5.004610915360639e-27
7
5.022175269941572e-27
8
5.0319529720528885e-27
9
5.106767851345048e-27
10
5.135528382605792e-27
11
5.268440817448297e-27
12
5.39626763947349e-27
13
5.427006299793089e-27
14
5.497205954921645e-27
15
5.509527142368529e-27
16
5.52519185891536e-27
17
5.555453764580273e-27
18
5.573873324279865e-27
19
5.6396642852670215e-27
20
5.6396642852670215e-27
21
5.678852274375825e-27
22
5.7227170582186176e-27
23
5.828509896033081e-27
24
5.832530778122645e-27
25
5.8591215345173054e-27
26
5.914199521757464e-27
27
5.9400077903577306e-27
28
5.971978176812189e-27
29
6.031784931565643e-27
30
6.0507170757117494e-27
31
6.088476483887127e-27
32
6.1253073943871975e-27
33
6.194968402146063e-27
34
6.2247445521095946e-27
35
6.264825545894184e-27
36
6.272701353392202e-27
37
6.307706481083736e-27
38
6.3377636977096986e-27
39
6.361364312571249e-27


In [ ]:
temp2.fitness_calculate([],[])
sorted_population2 = sorted(temp2.population, key = lambda test_suite: test_suite.fitness_value)   
with open('ga_cit.txt', 'w') as f:
    f.write(str(sorted_population2[-1].additional))
print(len(sorted_population2[-1].additional))